In [2]:
import torch

from torch.nn import Module
from torchtext.data import Field, LabelField, NestedField, Dataset, BucketIterator

In [5]:
import pandas as pd

In [6]:
# exp_data = {"text": ["đây là text thử nghiệm", "đây là text thử nghiệm lần 2"], "label": [0, 1]}
# df = pd.DataFrame(exp_data)
# df.to_csv("experiment.csv", header=None, index=False)

In [12]:
input_word = Field(init_token="<bos>", eos_token="<eos>", batch_first=True)
input_char_nesting = Field(tokenize=list, init_token="<bow>", eos_token="<eow>", batch_first=True)
input_char = NestedField(input_char_nesting, init_token="<bos>", eos_token="<eos>")
labels = LabelField(sequential=False)

/home/giangvl/topcv/Spell Checker/hybrid-transformer/venv/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/giangvl/topcv/Spell Checker/hybrid-transformer/venv/lib/python3.8/site-packages/torchtext/data/field.py:498: UserWarning: NestedField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/giangvl/topcv/Spell Checker/hybrid-transformer/venv/lib/python3.8/site-packages/torchtext/

In [13]:
fields = ([(('input_word', 'input_char'), (input_word, input_char)), ('labels', labels)])

In [14]:
from torchtext.data import TabularDataset

In [15]:
dataset = TabularDataset(path="experiment.csv", format="CSV", fields=fields)

/home/giangvl/topcv/Spell Checker/hybrid-transformer/venv/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/home/giangvl/topcv/Spell Checker/hybrid-transformer/venv/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [29]:
labels.build_vocab(dataset, max_size=2, min_freq=1)

In [28]:
dataset.examples[0].labels

'0'

In [21]:
from torchtext.data import Iterator

In [22]:
dataset[0].__dict__.keys()

dict_keys(['input_word', 'input_char', 'labels'])

In [23]:
input_char.build_vocab(dataset, max_size=10, min_freq=1)

In [24]:
input_word.build_vocab(dataset, max_size=10, min_freq=1)

In [25]:
dataset.examples[0].input_char

[['đ', 'â', 'y'],
 ['l', 'à'],
 ['t', 'e', 'x', 't'],
 ['t', 'h', 'ử'],
 ['n', 'g', 'h', 'i', 'ệ', 'm']]

In [30]:
data_iterator = Iterator(dataset, batch_size=2)

/home/giangvl/topcv/Spell Checker/hybrid-transformer/venv/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: Iterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [37]:
for item in data_iterator:
    print(item.input_char.view(2, 72))

tensor([[ 4,  2,  5,  1,  1,  1,  1,  1,  4,  0,  0, 15,  5,  1,  1,  1,  4,  8,
          0,  5,  1,  1,  1,  1,  4,  6, 10, 14,  6,  5,  1,  1,  4,  6,  7,  0,
          5,  1,  1,  1,  4,  9, 11,  7, 12,  0, 13,  5,  4,  8,  0,  9,  5,  1,
          1,  1,  4,  0,  5,  1,  1,  1,  1,  1,  4,  3,  5,  1,  1,  1,  1,  1],
        [ 4,  2,  5,  1,  1,  1,  1,  1,  4,  0,  0, 15,  5,  1,  1,  1,  4,  8,
          0,  5,  1,  1,  1,  1,  4,  6, 10, 14,  6,  5,  1,  1,  4,  6,  7,  0,
          5,  1,  1,  1,  4,  9, 11,  7, 12,  0, 13,  5,  4,  3,  5,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1]])


In [32]:
from transformers import AlbertConfig, AlbertModel

In [33]:
albertconfig = AlbertConfig()

In [34]:
model = AlbertModel(albertconfig)

In [39]:
for item in data_iterator:
    a = model(item.input_char.view(2, 72))
    print(a)

BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 0.1412,  0.4174, -0.5939,  ..., -1.9761,  1.0480,  0.1583],
         [ 0.1416,  0.4182, -0.5938,  ..., -1.9740,  1.0482,  0.1587],
         [ 0.1410,  0.4183, -0.5929,  ..., -1.9744,  1.0484,  0.1576],
         ...,
         [ 0.1422,  0.4177, -0.5928,  ..., -1.9753,  1.0478,  0.1581],
         [ 0.1417,  0.4177, -0.5931,  ..., -1.9752,  1.0475,  0.1587],
         [ 0.1414,  0.4182, -0.5935,  ..., -1.9759,  1.0478,  0.1577]],

        [[ 0.0711,  0.4130, -0.6131,  ..., -2.0495,  1.0349,  0.2711],
         [ 0.0715,  0.4138, -0.6130,  ..., -2.0473,  1.0353,  0.2712],
         [ 0.0709,  0.4141, -0.6121,  ..., -2.0476,  1.0356,  0.2702],
         ...,
         [ 0.0720,  0.4135, -0.6121,  ..., -2.0485,  1.0347,  0.2708],
         [ 0.0715,  0.4135, -0.6124,  ..., -2.0483,  1.0345,  0.2712],
         [ 0.0713,  0.4140, -0.6129,  ..., -2.0490,  1.0347,  0.2705]]],
       grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[-0.72

In [42]:
b = a.last_hidden_state.view(2, 9, 8, 4096)

In [46]:
b.mean(dim=1).shape

torch.Size([2, 8, 4096])